In [222]:
import random
from  copy import deepcopy
import math

In [ ]:
class city:
    def __init__(self,loc,num):
        self.loc=loc
        self.num=num


 
    def __eq__(self, other: object) -> bool:
        if other is None:
            return False  # 如果 other 是 None，直接返回 False
        if isinstance(other, city):
            return self.loc == other.loc and self.num == other.num  # 只在 other 是 city 类型时进行比较
        return False  # 如果 other 不是 city 类型，也返回 False
    def __str__(self) -> str:
        return f"{self.loc}, {self.num}"

a=city((1,2),6)
print(a)


(1, 2), 6


In [224]:
class path:
    def __init__(self,length,line=None):
        self.line=line
        self.len=length
        self.all=None
    def __eq__(self,other):
        return self.line==other.line
    def __hash__(self) -> int:
        return hash(tuple(self.line))
    def print_path(self):
        return f"{self.line}"

    
    # 交叉
    def intersect(self, other):
        # 随机选择两个交叉点
        i, j = random.sample(range(self.len), 2)
        if i >= j:
            i, j = j, i  # 确保 i < j

        # 创建新的路径，填充交叉部分
        new_line = [None] * self.len
        new_line[i:j] = self.line[i:j]

        # 当前插入的位置，初始化为 j
        current_index = j

        # 将 other.line 中未出现在 new_line 中的城市插入
        for city in other.line:
            # 如果城市已经在 new_line 中，跳过
            if city not in new_line:
                # 找到空位插入城市
                while new_line[current_index % self.len] is not None and current_index-j<self.len:
                    current_index += 1

                new_line[current_index % self.len] = city

        return path(self.len, new_line)


    # 变异
    def mutate(self):
        i, j = random.sample(range(self.len), 2)
        new_path = deepcopy(self)
        new_path.line[i], new_path.line[j] = new_path.line[j], new_path.line[i]
        return new_path



    def dis(self):
        distance = 0
        for i in range(self.len):
            if self.line[i] is None or self.line[(i + 1) % self.len] is None:
                continue  # 如果某个城市是 None，跳过
            sub_dis = math.dist(tuple(self.line[i].loc), tuple(self.line[(i + 1) % self.len].loc))
            distance += sub_dis
        self.all = distance

  
    





In [ ]:
import random
import math
from copy import deepcopy

class GeneticAlgorithm:
    def __init__(self, population_size, crossover_rate, mutation_rate, generations):
        self.population_size = population_size
        self.crossover_rate = crossover_rate
        self.mutation_rate = mutation_rate
        self.generations = generations
        self.population = []

    def initialize_population(self, path_length):
        cities = [city((random.randint(0, 100), random.randint(0, 100)), i) for i in range(path_length)]
        for i in cities:
            print(i)
        for _ in range(self.population_size):
            
            random.shuffle(cities)
            self.population.append(path(len(cities), cities))

    # 总体适应数量，作为除数    
    def calculate_fitness(self):
        fitness_values = []
        for individual in self.population:
            individual.dis()
            fitness_values.append(1 / (individual.all + 1e-6))  # 防止出现0   dis越大，占比越小
        return fitness_values

# 轮盘选择
    def roulette_wheel_selection(self, fitness_values):
        total_fitness = sum(fitness_values)
        probabilities = [f / total_fitness for f in fitness_values]
        cumulative_probabilities = [sum(probabilities[:i + 1]) for i in range(len(probabilities))]
        
        # 确保累积概率的最后一个值是 1（避免浮点误差）
        cumulative_probabilities[-1] = 1.0
        
        selected = []
        for _ in range(2): 
            r = random.random()
            selected_parent = None 
    
            
            for i, cp in enumerate(cumulative_probabilities):

                if r <= cp:
                    selected_parent = self.population[i]
                    selected.append(selected_parent)
                    print(f"Selected parent: {i} (City {selected_parent.line[0].num})")
                    break
            
   
            if selected_parent is None:
   
                break  
            
        return selected


    def evolve(self):
        for generation in range(self.generations):
            new_population = []
           
            fitness_values = self.calculate_fitness()

            for _ in range(self.population_size // 2):
                # 选择
                parent1, parent2 = self.roulette_wheel_selection(fitness_values)
            
                # Crossover
            
                if random.random() < self.crossover_rate:
                    child1 = parent1.intersect(parent2)
                    child2 = parent2.intersect(parent1)
                else:
                    child1, child2 = deepcopy(parent1), deepcopy(parent2)
        
                # Mutate
                if random.random() < self.mutation_rate:
                    child1 = child1.mutate()
                if random.random() < self.mutation_rate:
                    child2 = child2.mutate()

                new_population.extend([child1, child2])
           

            self.population = new_population
           
            
            best_fitness = max(self.calculate_fitness())
            print(f"Generation {generation + 1}, Best Fitness: {best_fitness:.6f}")

    def get_best_solution(self):

        best_individual = min(self.population, key=lambda p: p.all)
        return best_individual


if __name__ == "__main__":
    ga = GeneticAlgorithm(population_size=5, crossover_rate=0.8, mutation_rate=0.1, generations=5)
    ga.initialize_population(path_length=5)  
    print(ga.population[0])

    ga.evolve()

    best_solution = ga.get_best_solution()
    print("Best Path:", [city.num for city in best_solution.line])
    print("Shortest Distance:", best_solution.all)

        



(27, 87), 0
(81, 49), 1
(13, 71), 2
(50, 63), 3
(70, 63), 4
Selected parent: 4 (City 2)
Selected parent: 4 (City 2)
Selected parent: 3 (City 2)
Selected parent: 3 (City 2)
Generation 1, Best Fitness: 0.006102
Selected parent: 2 (City 2)
Selected parent: 0 (City 2)
Selected parent: 3 (City 2)
Selected parent: 1 (City 4)
Generation 2, Best Fitness: 0.004706
Selected parent: 0 (City 3)
Selected parent: 1 (City 2)
Selected parent: 3 (City 3)
Selected parent: 0 (City 3)
Generation 3, Best Fitness: 0.005102
Selected parent: 2 (City 2)
Selected parent: 2 (City 2)
Selected parent: 3 (City 0)
Selected parent: 2 (City 2)
Generation 4, Best Fitness: 0.004088
Selected parent: 0 (City 2)
Selected parent: 3 (City 2)
Selected parent: 1 (City 2)
Selected parent: 3 (City 2)
Generation 5, Best Fitness: 0.006106
Best Path: [4, 3, 0, 2, 1]
Shortest Distance: 163.77659925824602
